In [ ]:
pip install line_profiler


In [ ]:
%load_ext line_profiler

In [ ]:
# coding = 'utf-8'
import numpy as np
import pandas as pd
import timeit

from collections import defaultdict
from line_profiler import LineProfiler


In [ ]:

def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])

    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [ ]:
def target_mean_v2(data, y_name, x_name):

    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()

    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [ ]:
def target_mean_v3(data, y_name, x_name):

    result = np.zeros(data.shape[0])
    x = data[x_name].values
    y = data[y_name].values

    value_dict = defaultdict(lambda:0)
    count_dict = defaultdict(lambda:0)

    for i,j in zip(x,y):
         value_dict[i] += j
         count_dict[i] += 1
         result = [(value_dict[i] - j)/(count_dict[i] - 1) for i,j in zip(x,y)]
    return result

In [ ]:
def target_mean_v4(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
    data_shape = data.shape[0]
    result = np.zeros(data_shape)
    value_dict = dict()
    count_dict = dict()

    for i in range(data_shape):
        data_loc_x = data.loc[i, x_name]
        data_loc_y = data.loc[i, y_name]
        if data_loc_x not in value_dict:
            value_dict[data_loc_x] = data_loc_y
            count_dict[data_loc_x] = 1
        else:
            value_dict[data_loc_x] += data_loc_y
            count_dict[data_loc_x] += 1
    for i in range(data_shape):
        data_loc_x = data.loc[i, x_name]
        data_loc_y = data.loc[i, y_name]
        result[i] = (value_dict[data_loc_x] - data_loc_y) / (count_dict[data_loc_x] - 1)
    return result

In [ ]:
def target_mean_v5(data:pd.DataFrame, y_name:str, x_name:str) -> np.ndarray:
    data_shape = data.shape[0]
    result = np.zeros(data_shape)
    value_dict = dict()
    count_dict = dict()

    x_val_series = data.loc[:, x_name].values
    y_val_series = data.loc[:, y_name].values
    for i in range(data_shape):
        data_loc_x = x_val_series[i]
        data_loc_y = y_val_series[i]
        if data_loc_x not in value_dict:
            value_dict[data_loc_x] = data_loc_y
            count_dict[data_loc_x] = 1
        else:
            value_dict[data_loc_x] += data_loc_y
            count_dict[data_loc_x] += 1
    for i in range(data_shape):
        data_loc_x = x_val_series[i]
        data_loc_y = y_val_series[i]
        result[i] = (value_dict[data_loc_x] - data_loc_y) / (count_dict[data_loc_x] - 1) 

    return result

In [ ]:
y = np.random.randint(2, size=(100000, 1))
x = np.random.randint(10, size=(100000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [ ]:
def main():
    y = np.random.randint(2, size=(5000, 1))
    x = np.random.randint(10, size=(5000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

    # result_1 = target_mean_v1(data, 'y', 'x')
    # result_2 = target_mean_v2(data, 'y', 'x')
    # diff = np.linalg.norm(result_1 - result_2)
    # print(diff)

    profile = LineProfiler(target_mean_v2) #把函数传递到性能分析器
    profile.enable() #开始分析
    target_mean_v2(data, 'y', 'x')
    profile.disable() #停止分析
    profile.print_stats() #打印出性能分析结果


if __name__ == '__main__':
    main()

In [ ]:
%lprun -f target_mean_v1 target_mean_v1(data, 'y', 'x')

In [ ]:
%%timeit
target_mean_v1(data, 'y', 'x')


In [ ]:
%lprun -f target_mean_v2 target_mean_v2(data, 'y', 'x')

In [ ]:
%lprun -f target_mean_v3 target_mean_v3(data, 'y', 'x')

In [ ]:
%lprun -f target_mean_v4 target_mean_v4(data, 'y', 'x')